##Pips

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
%%capture
!pip install sentencepiece
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nlpaug
!pip install git+https://github.com/csebuetnlp/normalizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 KB 12.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-d2k79phn
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-d2k79phn
  Resolved https://github.com/csebuetnlp/normalizer to commit d80c3c484e1b80268f2b2dfaf7557fe65e34f321
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 KB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6883 sha256=5ace41df9a7962ae2524cd0bc265aed997e8

## Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

from transformers import *


In [ ]:
from normalizer import normalize
import nlpaug.augmenter.word.context_word_embs as aug

We will use weights and biases for tracking experiments and runs. Project page : https://wandb.ai/tasmiah-tahsin/fake-news-blurr

##function definitions

In [ ]:
def calc_len(sent):
  word_count = len(sent.split())
  return word_count

##Loading data

Load your DS in this section. You may need to modify this section according to your DS attributes. Also note that there are two sub sections - initial iteration and next iterations. This is because we had to augment whole DS in batches as google collab runs for a limited time. 

Run the initial iteration only the first time. And do not run "next iteration" in first time.

In each subsequent times, do not run initial iteration, run only the next iteration. After the main code section, an output is generated of how much data has been augmented.

###initial iteration
don't run this sub-section after initial iteration

In [ ]:
path1 = '/content/drive/MyDrive/datasets/Fake-1K.csv'

In [ ]:
df = pd.read_csv(path1)
df.head(2)

,articleID,domain,date,category,headline,content,label
0,1,channeldhaka.news,2019-03-14T13:34:14+00:00,International,মুরগির হামলায় শেয়াল নিহত,"বাংলায় একটা প্রবাদ আছে, শেয়ালের কাছে মুরগী বর্...",0
1,2,earki.com,"সেম্বর ১৭, ২০১৮",Miscellaneous,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম,"BTV থেকে লোকজন আসছে, ইন্টারভিউ নিবে।চারজনের টি...",0


In [ ]:
df['text'] = df['headline'] + '। ' + df['content']
df = df.drop(['headline','content'],axis=1)
df.head(2)

,articleID,domain,date,category,label,text
0,1,channeldhaka.news,2019-03-14T13:34:14+00:00,International,0,মুরগির হামলায় শেয়াল নিহত। বাংলায় একটা প্রবাদ আ...
1,2,earki.com,"সেম্বর ১৭, ২০১৮",Miscellaneous,0,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম। BTV থ...


In [ ]:
split1 = df
split1['text'][0]

'মুরগির হামলায় শেয়াল নিহত। বাংলায় একটা প্রবাদ আছে, শেয়ালের কাছে মুরগী বর্গা দেওয়া। প্রবাদটা এজন্য বলা হয় যে, শিয়ালের কাজই হলো মুরগী খেয়ে ফেলা। শেয়ালের কাছ থেকে মুরগী কখনো নিজেকে রক্ষা করতে পারেনা।তবে ফ্রান্সের এক বিশ্ববিদ্যালয়ের শিক্ষক-শিক্ষার্থীরা জানাচ্ছে, এবার এক আশ্চর্য্যজনক ঘটনা ঘটছে। মুরগীর হামলায় শিয়াল মারা গেছে।ঘটনাটি গত সপ্তাহের, ফ্রান্সের উত্তর-পূর্বাঞ্চলের বুতানিয়া এলাকার একটি কৃষিবিষয়ক বিদ্যালয়ে। ওই বিদ্যালয়ে একটি মুরগির খামার রয়েছে। মুরগির ঘরের এক কোনায় সকালে মৃত একটি শিয়াল পড়ে থাকতে দেখে শিক্ষার্থীরা।৬ হাজার মুরগি রয়েছে খামারটিতে। শিয়ালটির মৃতদেহ যেখানে পাওয়া গেছে, সেখানে তিন হাজার মুরগি ছিল। মুরগিগুলো সারা দিন বাইরে চরে বেড়ায়। সন্ধ্যা হলে নিজ থেকে ঘরে উঠে আসে। সূর্য ডুবে গেলেই দরজা বন্ধ হয়ে যায় স্বয়ংক্রিয়ভাবে।স্থানীয় বন্য প্রাণী বিশেষজ্ঞরা জানিয়েছেন, এই ঘটনায় তাঁরা অবাক। শিয়ালটি বাচ্চা ও অনভিজ্ঞ ছিল। এতগুলো মুরগির সামনে পড়ে সে সম্ভবত ভড়কে গিয়েছিল।Source- BBC Bangla'

In [ ]:
a_sent = ""
for i in range(len(split1)):
  sent = split1['text'][i]
  if(calc_len(sent)<200):
    a_sent = sent
    print(i,' ', a_sent)
    break;


0   মুরগির হামলায় শেয়াল নিহত। বাংলায় একটা প্রবাদ আছে, শেয়ালের কাছে মুরগী বর্গা দেওয়া। প্রবাদটা এজন্য বলা হয় যে, শিয়ালের কাজই হলো মুরগী খেয়ে ফেলা। শেয়ালের কাছ থেকে মুরগী কখনো নিজেকে রক্ষা করতে পারেনা।তবে ফ্রান্সের এক বিশ্ববিদ্যালয়ের শিক্ষক-শিক্ষার্থীরা জানাচ্ছে, এবার এক আশ্চর্য্যজনক ঘটনা ঘটছে। মুরগীর হামলায় শিয়াল মারা গেছে।ঘটনাটি গত সপ্তাহের, ফ্রান্সের উত্তর-পূর্বাঞ্চলের বুতানিয়া এলাকার একটি কৃষিবিষয়ক বিদ্যালয়ে। ওই বিদ্যালয়ে একটি মুরগির খামার রয়েছে। মুরগির ঘরের এক কোনায় সকালে মৃত একটি শিয়াল পড়ে থাকতে দেখে শিক্ষার্থীরা।৬ হাজার মুরগি রয়েছে খামারটিতে। শিয়ালটির মৃতদেহ যেখানে পাওয়া গেছে, সেখানে তিন হাজার মুরগি ছিল। মুরগিগুলো সারা দিন বাইরে চরে বেড়ায়। সন্ধ্যা হলে নিজ থেকে ঘরে উঠে আসে। সূর্য ডুবে গেলেই দরজা বন্ধ হয়ে যায় স্বয়ংক্রিয়ভাবে।স্থানীয় বন্য প্রাণী বিশেষজ্ঞরা জানিয়েছেন, এই ঘটনায় তাঁরা অবাক। শিয়ালটি বাচ্চা ও অনভিজ্ঞ ছিল। এতগুলো মুরগির সামনে পড়ে সে সম্ভবত ভড়কে গিয়েছিল।Source- BBC Bangla


###next iterations

In [ ]:
path2 = '/content/drive/MyDrive/datasets/fake_aug_csebuetnlp.csv'
split1 = pd.read_csv(path2)

In [ ]:
split1['text'][0]

"['মুরগির হামলায় শেয়াল নিহত। বাংলায় একটা প্রবাদ আছে, শেয়ালের কাছে মুরগী বর্গা দেওয়া। প্রবাদটা এজন্য বলা হয় যে, শিয়ালের কাজই হলো মুরগী খেয়ে ফেলা। শেয়ালের কাছ থেকে মুরগী কখনো নিজেকে রক্ষা করতে পারেনা।তবে ফ্রান্সের একটা বিশ্ববিদ্যালয়ের শিক্ষক-শিক্ষার্থীরা জানাচ্ছে, এবার এক আশ্চর্য্যজনক ঘটনা ঘটছে। মুরগীর হামলায় শিয়াল মারা গেছে।ঘটনাটি গত সপ্তাহের, ফ্রান্সের উত্তর-পূর্বাঞ্চলের বুতানিয়া এলাকার শহরের কৃষিবিষয়ক বিদ্যালয়ে। ওই বিদ্যালয়ে একটি মুরগির খামার রয়েছে। মুরগির ঘরের এক কোনায় সকালে মৃত একটি শিয়াল পড়ে থাকতে দেখে শিক্ষার্থীরা। কয়েক হাজার মুরগি রয়েছে খামারটিতে। শিয়ালটির মৃতদেহ যেখানে পাওয়া গেছে, সেখানে সাত হাজার মুরগি ছিল। মুরগিগুলো সারা দিন একসাথে চরে বেড়ায়। সন্ধ্যা হলে নিজ থেকে ঘরে বাইরে আসে। সূর্য ডুবে গেলেই দরজা বন্ধ হয়ে যায়  ওদের।স্থানীয় বন্য প্রাণী বিশেষজ্ঞরা জানিয়েছেন, এই ঘটনায় তাঁরা অবাক। শিয়াল আসলে বাচ্চা ও অনভিজ্ঞ ছিল। এতগুলো মুরগির সামনেই সে সম্ভবত ভড়কে গিয়েছিল। দেখুন- bbc bangla']"

##Data Augmentation

###load augmenter and an example

In [ ]:
# !pip install git+https://github.com/csebuetnlp/normalizer
# !pip install nlpaug
# from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
#import nlpaug.augmenter.word.context_word_embs as aug

In [ ]:
augmenter = aug.ContextualWordEmbsAug(model_path='csebuetnlp/banglabert_generator', action="substitute")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglabert_generator/snapshots/6b36cf4fab487a50c1a44d7f12af563c75a9a846/config.json
Model config ElectraConfig {
  "_name_or_path": "csebuetnlp/banglabert_generator",
  "architectures": [
    "ElectraForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglabert_generator/snapshots/6b36cf4fab487a50c1a44d7f12af563c75a9a846/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglabert_generator/snapshots/6b36cf4fab487a50c1a44d7f12af563c75a9a846/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglabert_generator/snapshots/6b36cf4fab487a50c1a44d7f12af563c75a9a846/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--csebuetnlp--banglabert_generator/snapshots/6b36cf4fab487a50c1a44d7f12af563c75a9a846/config.json
Model config ElectraConfig {
  "_name_or_path": "csebuetnlp/banglabert_generator",
  "architectures": [
    "ElectraForMaskedLM"
  ],
  "attention_probs_dropout_

In [ ]:
a_sent = 'মুরগির হামলায় শেয়াল নিহত'

In [ ]:
output = print(augmenter.augment(normalize(a_sent)))
output

['মুরগির অভিযানে শেয়াল নিহত ৪']


In [ ]:
output = print(augmenter.augment(normalize(a_sent),n=2))


['মুরগির পেটে হামলায় শেয়াল নিহত । আজ বাংলায় একটা প্রবাদ আছে, শেয়ালের কাছে মুরগী বর্গা দেওয়া । প্রবাদটা এজন্য বলা হয় যে, শিয়ালের কাজই হলো মুরগী খেয়ে ফেলা । শেয়ালের বাহিনীর কাছ থেকে মুরগী কখনো নিজেকে রক্ষা করতে পারেনা । তবে ফ্রান্সের এক বিশ্ববিদ্যালয়ের শিক্ষক - শিক্ষার্থীরা জানাচ্ছে, এবার এক আশ্চর্য্যজনক ঘটনা ঘটছে । মুরগীর হামলায় শিয়াল মারা গেছে । ঘটনাটি গত সপ্তাহের, ফ্রান্সের উত্তর - পূর্বাঞ্চলের তেল বুতানিয়া এলাকার পাশের একটি কৃষিবিষয়ক বিদ্যালয়ে । তবে ওই বিদ্যালয়ে একটি মুরগির খামার রয়েছে । মুরগির ঘরের এক কোনায় সকালে মৃত একটি শিয়াল পড়ে থাকতে দেখে শিক্ষার্থীরা । ৬ হাজার মুরগি রয়েছে খামারটিতে । শিয়ালটির মৃতদেহ সকালে যেখানে পাওয়া গেছে, সেখানে তিন হাজার মুরগি ছিল । মূলত মুরগিগুলো সারা দিন বাইরে চরে বেড়ায় । সন্ধ্যা হলে নিজ থেকে ঘরে উঠে আসে । সূর্য ডুবে গেলেই সবগুলো দরজা বন্ধ হয়ে যায় স্বয়ংক্রিয়ভাবে । স্থানীয় বন্য প্রাণী বিশেষজ্ঞরা জানিয়েছেন, এই ঘটনায় তাঁরা অবাক । শিয়ালটি বাচ্চা খেলা ও অনভিজ্ঞ ছিল । এতগুলো মুরগির সামনে পড়ে সে সম্ভবত ভড়কে গিয়েছিল । Source - B

In [ ]:
stnumr = 10

In [ ]:
type(output)

NoneType

###batch augment whole DS

In [ ]:
split1.head()

,articleID,domain,date,category,label,text
0,1,channeldhaka.news,2019-03-14T13:34:14+00:00,International,0,মুরগির হামলায় শেয়াল নিহত। বাংলায় একটা প্রবাদ আ...
1,2,earki.com,"সেম্বর ১৭, ২০১৮",Miscellaneous,0,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম। BTV থ...
2,3,earki.com,"২০:৩৯, জানুয়ারি ১৪, ২০১৯",Miscellaneous,0,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...
3,4,channeldhaka.news,2018-06-30T15:56:47+00:00,Sports,0,অবসর নেয়ার ঘোষণা দিলেন মেসি !। রাশিয়া বিশ্বকাপ...
4,5,motikontho.wordpress.com,2013-03-05T21:55:45+00:00,Miscellaneous,0,"মাদারফাকার নহে, ব্রাদারফাকার: সাকা | দৈনিক মতি..."


In [ ]:
split1.shape

(1299, 6)

In [ ]:
#use start and end variables to control how much of your DS you want to augment. The output shown after this section 
#shows how much has been successfully converted and saved in you gdrive. 
start = 0
end = len(split1)
for i in range(start, end):
  sent = split1['text'][i]
 
  res_sent = augmenter.augment(normalize(sent))
  
  split1.at[i, 'text'] = res_sent
  
  #on every 50th row, making a save
  if(i%50==0):
    split1.to_csv('/content/drive/MyDrive/datasets/fake_aug_csebuetnlp.csv', index=False)
    print(i)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250


In [ ]:
split1['text'][0]

['মুরগির হামলায় শেয়াল নিহত । বাংলায় একটা প্রবাদ আছে, শেয়ালের কাছে মুরগী বর্গা দেওয়া । প্রবাদটা এজন্য বলা উচিত যে, ইঁদুরের কাজই হলো মুরগী খেয়ে ফেলা । ইঁদুরের কাছ থেকে মুরগী কখনো নিজেকে রক্ষা করতে পারেনা । তবে ফ্রান্সের এক বিশ্ববিদ্যালয়ের শিক্ষক - শিক্ষার্থীরা জানাচ্ছে, এবার একটা আশ্চর্য্যজনক ঘটনা ঘটছে যখন মুরগীর হামলায় শিয়াল মারা গেছে । ঘটনাটি গত সপ্তাহের, ফ্রান্সের উত্তর - পূর্বাঞ্চলের বুতানিয়া এলাকার একটি কৃষিবিষয়ক বিদ্যালয়ে । ওই মাঠেই একটি মুরগির খামার রয়েছে । মুরগির ঘরের এক কোনায় সকালে মৃত একটি শিয়াল পড়ে থাকতে দেখে শিক্ষার্থীরা । ৬ হাজার মুরগি রয়েছে খামারটিতে । শিয়ালটির মৃতদেহ যেখানে পাওয়া গেছে, সেখানে তিন হাজার মুরগি ঝুলছে । মুরগিগুলো সারা দিন বাইরে চরে বেড়ায় । সন্ধ্যা হলে নিজ থেকে ঘরে উঠে আসে । সূর্য ডুবে গেলেই দরজা বন্ধ হয়ে যায় স্বয়ংক্রিয়ভাবে বলেই স্থানীয় স্থানীয় প্রাণী বিশেষজ্ঞরা জানিয়েছেন, এই ঘটনায় তাঁরা অবাক । শিয়ালটি বাচ্চা ও অনভিজ্ঞ ছিল । এতগুলো মুরগির সামনে থেকে সে সম্ভবত ভড়কে গিয়েছিল । Source - BBC Bangla']

In [ ]:
#done. save all.
split1.to_csv('/content/drive/MyDrive/datasets/fake_aug_csebuetnlp.csv', index=False)